## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

# \[Backup Queries\]

> **Useful Queries**
> 
> - Last backup times (Q1)
> - Backup Size(Q2)
> - Backup Duration(Q3)
> - Backup stats - copy only, time started/completed (Q4)
> - Last 30 days backup locations (Q5)
> - Are backups encypted (Q6)
> - Are backups in full access shares (Q7)
> - How many days backup history are kept (Q8)
> - Date database last restored (Q9)
> - Detailed Database backup/restore info (Q10)

## Last backup - log and database for each database (Q1)

In [ ]:
/* Last backup times */
SELECT database_name
	,CASE 
		WHEN [TYPE] = 'D'
			THEN 'Database'
		WHEN [TYPE] = 'I'
			THEN 'Differential database'
		WHEN [TYPE] = 'L'
			THEN 'LOG'
		WHEN [TYPE] = 'F'
			THEN 'FILE'
		ELSE 'OTHER'
		END AS [TYPE]
	,MAX(backup_finish_date) AS [lastbackup_finish_date]
FROM msdb..backupset
WHERE [TYPE] = 'D'
	OR [TYPE] = 'L'
	OR [TYPE] = 'I'
GROUP BY database_name
	,[TYPE]
ORDER BY database_name
	,MAX(backup_finish_date)

## Backup Sizes(Q2)

In [ ]:
/* Backup Sizes */
SELECT b.database_name 'Database Name'
	,CONVERT(BIGINT, b.backup_size / 1048576) 'UnCompressed Backup Size (MB)'
	,CONVERT(BIGINT, b.compressed_backup_size / 1048576) 'Compressed Backup Size (MB)'
	,CONVERT(NUMERIC(20, 2), (CONVERT(FLOAT, b.backup_size) / CONVERT(FLOAT, b.compressed_backup_size))) 'Compression Ratio'
	,DATEDIFF(SECOND, b.backup_start_date, b.backup_finish_date) 'Backup Elapsed Time (sec)'
FROM msdb.dbo.backupset b
WHERE DATEDIFF(SECOND, b.backup_start_date, b.backup_finish_date) > 0
	AND b.backup_size > 0
ORDER BY b.backup_finish_date DESC

IF (
		SELECT OBJECT_ID('TempDB.dbo.#tempbackup')
		) IS NOT NULL
	DROP TABLE #tempbackup

CREATE TABLE #tempbackup (
	database_name NVARCHAR(128)
	,[TYPE] CHAR(1)
	,backup_finish_date DATETIME
	)


## Backup Duration (Q3)

In [ ]:
/* Backup Duration */
SET NOCOUNT ON
GO

IF EXISTS (
		SELECT NAME
		FROM tempdb..sysobjects
		WHERE NAME LIKE '#DatabasesBackupsDuration%'
		)
	DROP TABLE #DatabasesBackupsDuration
GO

CREATE TABLE #DatabasesBackupsDuration (
	ServerName VARCHAR(100) NULL
	,DBName VARCHAR(100) NULL
	,RecoveryModel VARCHAR(100) NULL
	,LastFullbackup DATETIME NULL
	,FullbackupDurationSec BIGINT NULL
	,DBStatus VARCHAR(100) NULL
	,
	)
GO

INSERT INTO #DatabasesBackupsDuration (
	ServerName
	,DBName
	)
SELECT convert(VARCHAR, serverproperty('ServerName'))
	,a.NAME
FROM master.dbo.sysdatabases a
WHERE a.NAME <> 'tempdb'

UPDATE #DatabasesBackupsDuration
SET LastFullbackup = b.backup_start_date
FROM #DatabasesBackupsDuration a
	,(
		SELECT database_name
			,max(backup_start_date) backup_start_date
		FROM msdb..backupset
		WHERE type = 'D'
		GROUP BY database_name
		) b
WHERE a.DBName = b.database_name

UPDATE #DatabasesBackupsDuration
SET RecoveryModel = convert(SYSNAME, DatabasePropertyEx(DBName, 'Recovery'))

UPDATE #DatabasesBackupsDuration
SET DBStatus = convert(SYSNAME, DatabasePropertyEx(DBName, 'Status'))

UPDATE d
SET d.FullbackupDurationSec = datediff(s, backup_start_date, backup_finish_date)
FROM #DatabasesBackupsDuration d
	,(
		SELECT database_name
			,max(backup_start_date) AS backup_start_date
			,max(backup_finish_date) AS backup_finish_date
		FROM msdb..backupset
		WHERE type = 'D'
		GROUP BY database_name
		) b
WHERE d.DBName = b.database_name
GO

SELECT *
FROM #DatabasesBackupsDuration
ORDER BY LastFullbackup
GO

SELECT CAST(SUM(FullbackupDurationSec) / 60 AS VARCHAR(100)) + ' Minutes' AS FullBackupTimeTotal
FROM #DatabasesBackupsDuration
GO

DROP TABLE #DatabasesBackupsDuration


## Backup stats - copy only, time started/completed (Q4)

In [ ]:
 /* Backup Stats - started / completed */
SELECT d.NAME
	,b.NAME
	,b.user_name
	,backup_start_date
	,backup_finish_date
	,DATEDIFF(mi, backup_start_date, backup_finish_date) Timttaken_MINS
	,backup_size
	,compressed_backup_size
	,server_name
	,is_copy_only
FROM master.sys.databases d
LEFT OUTER JOIN msdb.dbo.backupset b ON d.NAME = b.database_name
	AND b.type = 'D'
WHERE d.database_id NOT IN (
		2
		,3
		)
	AND backup_start_date > GETDATE() - 1
ORDER BY d.NAME
	,backup_start_date DESC


## Last 30 days backup locations (Q5)

In [ ]:
--Last month backups file locations 
SELECT DISTINCT database_name
	,REVERSE(SUBSTRING(REVERSE(physical_device_name), CHARINDEX('\', REVERSE(physical_device_name)) + 1, LEN(physical_device_name)))
FROM [msdb].[dbo].[backupset] a
JOIN [msdb].[dbo].[backupmediafamily] b ON a.media_set_id = b.media_set_id
WHERE backup_start_date > DATEADD(d, - 30, getdate())

## Are backups encrypted (Q6)

In [ ]:
--Are backups encrypted
SELECT DISTINCT d.NAME
	,is_encrypted
FROM master.sys.databases d
LEFT OUTER JOIN msdb.dbo.backupset b ON d.NAME = b.database_name
	AND b.type = 'D'
WHERE d.database_id NOT IN (
		2
		,3
		)
	AND backup_start_date > GETDATE() - 31

## Are backups in full access shares (Q7)

In [ ]:
-- Are backups in full access shares 
SELECT DISTINCT REVERSE(SUBSTRING(REVERSE(physical_device_name), CHARINDEX('\', REVERSE(physical_device_name)), LEN(physical_device_name)))
FROM msdb.dbo.backupmediafamily
ORDER BY 1 DESC

## How many days backup history (Q8)

In [ ]:
--How many daya backup History
SELECT TOP 1 backup_start_date
	,DATEDIFF(d, backup_start_date, GETDATE()) AS NoofDaysHistory
FROM msdb.dbo.backupset WITH (NOLOCK)
ORDER BY backup_start_date

## Last restore date (Q9)

In [ ]:
--Last restore date
SELECT [rs].[destination_database_name]
	,MAX([rs].[restore_date]) RestoreDate
FROM msdb.dbo.restorehistory rs
INNER JOIN msdb.dbo.backupset bs ON [rs].[backup_set_id] = [bs].[backup_set_id]
INNER JOIN msdb.dbo.backupmediafamily bmf ON [bs].[media_set_id] = [bmf].[media_set_id]
GROUP BY [rs].[destination_database_name]

## Detailed Database backup/restore info (Q10)
### Backuplocation,size,backup dates,types, page verify, owner

In [ ]:
--Database backup detailed (Q10)
GO

SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
GO

IF (
		SELECT OBJECT_ID('TempDB.dbo.#tempbackup')
		) IS NOT NULL
	DROP TABLE #tempbackup

CREATE TABLE #tempbackup (
	database_name NVARCHAR(128)
	,[TYPE] CHAR(1)
	,backup_finish_date DATETIME
	)

INSERT INTO #tempbackup
SELECT database_name
	,[TYPE]
	,MAX(backup_finish_date)
FROM msdb..backupset
GROUP BY database_name
	,[TYPE]

CREATE CLUSTERED INDEX IXc_#tempbackup ON #tempbackup (database_name)

IF (
		SELECT OBJECT_ID('TempDB.dbo.#tmp_sp_db_vardecimal_storage_format')
		) IS NOT NULL
	DROP TABLE #tmp_sp_db_vardecimal_storage_format

CREATE TABLE #tmp_sp_db_vardecimal_storage_format (
	dbname SYSNAME NULL
	,vardecimal_enabled VARCHAR(3) NULL
	)

CREATE CLUSTERED INDEX ixc_tmp_sp_db_vardecimal_storage_format ON #tmp_sp_db_vardecimal_storage_format (dbname)

IF EXISTS (
		SELECT o.object_id
		FROM sys.system_objects o
		WHERE o.NAME = N'sp_db_vardecimal_storage_format'
		)
BEGIN
	INSERT INTO #tmp_sp_db_vardecimal_storage_format
	EXEC sys.sp_db_vardecimal_storage_format
END

IF (
		SELECT OBJECT_ID('Tempdb.dbo.#DatabaseSizes')
		) IS NOT NULL
	DROP TABLE #DatabaseSizes
GO

CREATE TABLE #DatabaseSizes (
	SrvName SYSNAME
	,DatabaseName SYSNAME
	,FileName SYSNAME
	,PhysicalName SYSNAME
	,TotalSizeinMB DECIMAL(18, 6)
	,AvailableSpaceInMB DECIMAL(18, 6)
	,file_id INT
	,FilegroupName SYSNAME NULL
	,Type_desc VARCHAR(10)
	)

EXEC sp_MSForEachDB 'USE [?]  INSERT INTO #DatabaseSizes
SELECT @@Servername AS SrvName,
db_name() AS DatabaseName,
f.name AS [File Name] , f.physical_name AS [Physical Name], 
CAST((f.size/128.0) AS DECIMAL(18,6)) AS [Total Size in MB],
CAST(f.size/128.0 - CAST(FILEPROPERTY(f.name, ''SpaceUsed'') AS int)/128.0 AS DECIMAL(18,6)) AS [Available Space In MB], 
[file_id], 
fg.name AS [Filegroup Name],f.Type_desc
FROM sys.database_files AS f WITH (NOLOCK) 
LEFT OUTER JOIN sys.data_spaces AS fg WITH (NOLOCK) 
ON f.data_space_id = fg.data_space_id OPTION (RECOMPILE);
'

SELECT DISTINCT @@ServerName AS ServerName
	,dtb.NAME AS [Name]
	,CASE 
		WHEN bk.type = 'D'
			THEN 'FULL'
		WHEN bk.type = 'L'
			THEN 'LOG'
		WHEN bk.type = 'I'
			THEN 'DIFF'
		WHEN bk.type = 'F'
			THEN 'FILEGROUP'
		WHEN bk.type = 'P'
			THEN 'Partial'
		WHEN bk.type = 'Q'
			THEN 'Differential partial'
		ELSE ' -'
		END AS 'Type'
	,REVERSE(SUBSTRING(REVERSE(bkm.physical_device_name), CHARINDEX('\', REVERSE(bkm.physical_device_name)) + 1, LEN(bkm.physical_device_name))) AS 'BackupLocation'
	,dtb.state_desc
	,db.PhysicalName
	,db.TotalDataSizeinMB
	,db.AvailableDataSpaceInMB
	,db.TotalLogSizeinMB
	,db.AvailableLogSpaceInMB
	,CASE 
		WHEN dtb.recovery_model = 1
			THEN 'FULL'
		WHEN dtb.recovery_model = 2
			THEN 'BULK_LOGGED'
		WHEN dtb.recovery_model = 3
			THEN 'SIMPLE'
		END AS [RecoveryModel]
	,(
		SELECT backup_finish_date
		FROM #tempbackup
		WHERE TYPE = 'D'
			AND database_name = dtb.NAME
		) AS [LastBackupDate]
	,(
		SELECT backup_finish_date
		FROM #tempbackup
		WHERE TYPE = 'I'
			AND database_name = dtb.NAME
		) AS [LastDifferentialBackupDate]
	,(
		SELECT backup_finish_date
		FROM #tempbackup
		WHERE TYPE = 'L'
			AND database_name = dtb.NAME
		) AS [LastLogBackupDate]
	,dtb.compatibility_level AS 'CompatibilityLevel [100]'
	,dtb.create_date AS [CreateDate]
	,dtb.is_fulltext_enabled AS [IsFullTextEnabled]
	,CAST(CASE 
			WHEN dtb.NAME IN (
					'master'
					,'model'
					,'msdb'
					,'tempdb'
					)
				THEN 1
			ELSE dtb.is_distributor
			END AS BIT) AS [IsSystemObject]
	,SUSER_SNAME(dtb.owner_sid) AS [Owner]
	,df.physical_name AS [PrimaryFilePath]
	,dtb.collation_name AS [COLLATION]
	,dtb.page_verify_option_desc AS [PageVerify]
	,dtb.is_read_only AS [ReadOnly]
	,CAST(ISNULL(dtb.source_database_id, 0) AS BIT) AS [IsDatabaseSnapshot]
	,dtb.NAME AS [DatabaseName]
	,CAST(@@version AS VARCHAR(128)) AS 'Version'
	,is_copy_only
FROM master.sys.databases AS dtb
LEFT JOIN sys.master_files AS df ON df.database_id = dtb.database_id
	AND 1 = df.data_space_id
	AND 1 = df.file_id
LEFT JOIN sys.database_recovery_status AS drs ON drs.database_id = dtb.database_id
LEFT JOIN sys.database_mirroring AS dmi ON dmi.database_id = dtb.database_id
LEFT JOIN #tmp_sp_db_vardecimal_storage_format AS vardec ON dtb.NAME = vardec.dbname
LEFT JOIN (
	SELECT SrvName
		,DatabaseName
		,PhysicalName
		,SUM(CASE 
				WHEN Type_desc != 'LOG'
					THEN TotalSizeinMB
				ELSE 0
				END) AS TotalDataSizeinMB
		,SUM(CASE 
				WHEN Type_desc != 'LOG'
					THEN AvailableSpaceInMB
				ELSE 0
				END) AS AvailableDataSpaceInMB
		,SUM(CASE 
				WHEN Type_desc = 'LOG'
					THEN TotalSizeinMB
				ELSE 0
				END) AS TotalLogSizeinMB
		,SUM(CASE 
				WHEN Type_desc = 'LOG'
					THEN AvailableSpaceInMB
				ELSE 0
				END) AS AvailableLogSpaceInMB
	FROM #DatabaseSizes
	GROUP BY SrvName
		,DatabaseName
		,PhysicalName
	) db ON dtb.NAME = db.DatabaseName
LEFT OUTER JOIN [msdb].[dbo].[backupset] bk ON bk.database_name = dtb.NAME
LEFT OUTER JOIN [msdb].[dbo].[backupmediafamily] bkm ON bk.media_set_id = bkm.media_set_id
WHERE bkm.device_type <> 7 --Not virtual


## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)